In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
vogeldata = pd.read_csv('Data/Dataset1/birds.csv')

In [2]:
vogeldata.head()

,gen,sp,ssp,grp,en,rec,cnt,loc,lat,lon,...,method,url,file,file-name,length,date,uploaded,animal-seen,playback-used,q
0,Gallinago,gallinago,NaN,birds,Common Snipe,Gosse Hoekstra,Netherlands,"Tzummarum, Waadhoeke, Friesland",53.2602,5.5326,...,field recording,//xeno-canto.org/843222,https://xeno-canto.org/843222/download,XC843222-CS-221123-08H25M.mp3,0:24,2023-11-22,2023-11-22,yes,no,B
1,Gallinago,gallinago,NaN,birds,Common Snipe,Michel Veldt,Netherlands,"Bodegraven, Bodegraven-Reeuwijk, Zuid-Holland",52.0747,4.7632,...,field recording,//xeno-canto.org/771007,https://xeno-canto.org/771007/download,XC771007-Watersnip-roep---Bodegraven---2022-10...,0:19,2022-10-17,2022-12-23,no,no,B
2,Gallinago,gallinago,NaN,birds,Common Snipe,Michel Veldt,Netherlands,"Bodegraven, Bodegraven-Reeuwijk, Zuid-Holland",52.0747,4.7632,...,field recording,//xeno-canto.org/771006,https://xeno-canto.org/771006/download,XC771006-Watersnip-roep---Bodegraven---2022-09...,0:21,2022-09-22,2022-12-23,no,no,B
3,Gallinago,gallinago,NaN,birds,Common Snipe,Michel Veldt,Netherlands,"Bodegraven, Bodegraven-Reeuwijk, Zuid-Holland",52.0747,4.7632,...,field recording,//xeno-canto.org/770906,https://xeno-canto.org/770906/download,XC770906-Watersnip-roep---Bodegraven--2020-10-...,0:10,2020-10-04,2022-12-22,no,no,B
4,Gallinago,gallinago,NaN,birds,Common Snipe,Peter van Vlaardingen,Netherlands,"Wijk bij Duurstede, Utrecht",51.9839,5.3368,...,field recording,//xeno-canto.org/753978,https://xeno-canto.org/753978/download,XC753978-221003_2036_CommonSnipe_3h38_nfc.wav,0:23,2022-10-03,2022-10-05,no,no,B


# Data Preperation

In [3]:
vogelkopie = vogeldata[['url', 'file', 'file-name']].copy()

vogelkopie.count()

url          500
file         500
file-name    500
dtype: int64

# !Onderstaande niet runnen anders download je opnieuw alle files!

In [ ]:
import requests
import os

# Create a directory to save the audio files
output_dir = "audio_files"
os.makedirs(output_dir, exist_ok=True)

# Iterate over each row in the DataFrame
for index, row in vogelkopie.iterrows():
    audio_file = row['file']  # Get the audio file URL
    file_name = row['file-name']  # Get the file name

    # Define the full path to save the file
    file_path = os.path.join(output_dir, file_name)

    try:
        # Download the audio file
        response = requests.get(audio_file, stream=True)
        response.raise_for_status()  # Raise an error for bad responses

        # Save the file locally
        with open(file_path, 'wb') as audio_file:
            audio_file.write(response.content)

        print(f"Downloaded: {file_name}")
    except Exception as e:
        print(f"Failed to download {file_name}: {e}")

# !OOK DIT NIET RUNNEN ANDERS DOE JE ALLE FILES OPNIEUW OPSPLITSEN IN FRAGMENTEN VAN 2 SECONDEN!

In [ ]:
import librosa
import soundfile as sf
import os

# Directory containing the audio files
input_dir = "audio_files"
output_dir = "audio_fragments"

# Iterate through all audio files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".mp3") or file_name.endswith(".wav") or file_name.endswith(".MP3"):  # Process only audio files
        file_path = os.path.join(input_dir, file_name)

        # Load the audio file
        y, sr = librosa.load(file_path, sr=None)  # Load with the original sampling rate

        # Split the audio into 2-second fragments
        fragment_duration = 2 * sr  # 2 seconds in samples
        for i in range(0, len(y), fragment_duration):
            fragment = y[i:i + fragment_duration]  # Extract the fragment
            fragment_name = f"{os.path.splitext(file_name)[0]}_fragment_{i // fragment_duration}.wav"
            fragment_path = os.path.join(output_dir, fragment_name)

            # Save the fragment as a .wav file
            sf.write(fragment_path, fragment, sr)
            print(f"Created: {fragment_name}")

print("All audio files have been split into 2-second fragments.")

Alle gedownloade audiobestanden zijn opgesplitst in fragmenten van 2 seconden. Dit zijn allemaal .wav files.

### Aanmaken van CSV-file voor het clusteren van de data
Voordat wij kunnen beginnen met clusteren moeten de audiofragement gelabeled worden. Dit moet helaas met de hand gebeuren. Een aantal stappen hiervan worden hieronder uitgelegd.

Er is een dataframe aangemaakt genaamd **fragments_df** hierin zijn de fragmenten geplaatst en gelabeld met de Nederlandse namen van de vogels. Deze is uiteindelijk in een csv bestand gezet zodat we hier altijd bij kunnen. Deze heet 'fragments_data.csv'. Voor deze dataset moest er per fragment-titel gekeken worden naar wat er in de naam staat en hoe deze in de 'vogeldata' dataframe staat. En welke Nederlandse naam van de vogels er dus bij hoorde. Dit is te zien in onderstaande code.

In [ ]:
#fragments_df.loc[fragments_df['fragment'].str.contains('tureluur', case=False, na=False), 'labels'] = 'tureluur'

Een aantal van de fragmenten was niet duidelijk welke vogel erbij hoorden en deze zijn dus gedropt van het dataframe.

In [ ]:
#fragments_df = fragments_df.dropna(subset=['labels'])

Uiteindelijk is er een .csv file gemaakt van dit fragment zodat we er in onze ontwikkelomgeving overal bijkunnen.

In [10]:
#fragments_df.to_csv('Data/fragments_data.csv', index=False)

#### Het Fragments_data.csv file

In [13]:
fragmenten_df = pd.read_csv('Data/fragments_data.csv')
fragmenten_df.head()

,fragment,labels,label_numeric
0,05190.100417.watersniproepEempolder_fragment_0...,watersnip,0
1,05190.100417.watersniproepEempolder_fragment_1...,watersnip,0
2,05190.100417.watersniproepEempolder_fragment_2...,watersnip,0
3,05190.100417.watersniproepEempolder_fragment_3...,watersnip,0
4,05190.100417.watersniproepEempolder_fragment_4...,watersnip,0


In [15]:
en_counts = fragmenten_df['labels'].value_counts()

print(en_counts)

labels
tureluur                  1238
oeverloper                 861
witgat                     674
watersnip                  579
zwarte ruiter              476
grauwe franjepoot           24
geelpootruiter              23
steenloper                  17
steltloper                   8
amerikaanse oeverloper       2
Name: count, dtype: int64
